In [ ]:
!pip install transformers --quiet
!pip install datasets --quiet

     |████████████████████████████████| 4.2 MB 38.1 MB/s 
     |████████████████████████████████| 6.6 MB 18.2 MB/s 
     |████████████████████████████████| 86 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 47.7 MB/s 
     |████████████████████████████████| 346 kB 8.4 MB/s 
     |████████████████████████████████| 1.1 MB 65.6 MB/s 
     |████████████████████████████████| 140 kB 74.3 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
     |████████████████████████████████| 212 kB 67.5 MB/s 
     |████████████████████████████████| 127 kB 68.9 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
     |████████████████████████████████| 271 kB 61.8 MB/s 
     |████████████████████████████████| 144 kB 69.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is 

In [ ]:
get_ipython().kernel.do_shutdown(True)

{'restart': True, 'status': 'ok'}

In [ ]:
from datasets import load_dataset
import torch
from torch import nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AdamW
import time
from tqdm import tqdm

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
class CustomDataset(Dataset):
    
    def __init__(self, data, labels, tokenizer, max_len):
        self.data = data
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def get_dataset(self):
      input_ids = []
      attention_masks = []
      targets = []
      for (sent1, sent2, label) in zip(self.data['sent1'], self.data['sent2'], self.data['label']):
        if label not in self.labels.keys():
          continue
        target = self.labels[label]
        sent1_encoded = self.tokenizer.encode(sent1, add_special_tokens=False)
        sent2_encoded = self.tokenizer.encode(sent2, add_special_tokens=False)
        combined_encoded = [self.tokenizer.cls_token_id] + sent1_encoded \
                            + [self.tokenizer.sep_token_id] + sent2_encoded + [self.tokenizer.cls_token_id]
        attention_mask = [1] * (len(sent1_encoded) + len(sent2_encoded) + 3)
        padded_combined_encoded = combined_encoded + [1] * (self.max_len - len(combined_encoded))
        padded_attention_mask = attention_mask + [0] * (self.max_len - len(attention_mask))
        input_ids.append(padded_combined_encoded)
        attention_masks.append(padded_attention_mask)
        targets.append(target)
      input_ids = torch.tensor(input_ids)
      attention_masks = torch.tensor(attention_masks)
      targets = torch.tensor(targets)
      return TensorDataset(input_ids, attention_masks, targets)

def create_data_loader(data, labels, tokenizer, max_len, batch_size):
    dataset = CustomDataset(data, labels, tokenizer, max_len)
    return DataLoader(dataset.get_dataset(), batch_size = batch_size)

In [ ]:
def train(model, train_dataloader, validation_dataloader):
  optimizer = AdamW(model.parameters(), lr=1e-5)
  EPOCHS = 10
  for epoch in range(EPOCHS):
    start = time.time()
    model.train()
    losses = []
    correct_preds = 0
    for d in tqdm(train_dataloader):
      input_ids = d[0].to(device)
      attention_mask = d[1].to(device)
      targets = d[2].to(device)
      outputs = model(input_ids, attention_mask=attention_mask, labels=targets)
      _, preds = torch.max(outputs.logits, dim=1)
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      correct_preds += torch.sum(preds == targets)
      losses.append(loss.item())
    train_acc = correct_preds / len(data['train'])
    train_loss = np.mean(losses)
    model.eval()
    losses = []
    correct_preds = 0
    with torch.no_grad():
        for d in tqdm(validation_dataloader):
          input_ids = d[0].to(device)
          attention_mask = d[1].to(device)
          targets = d[2].to(device)
          outputs = model(input_ids, attention_mask=attention_mask, labels=targets)
          _, preds = torch.max(outputs.logits, dim=1)
          loss = outputs.loss
          correct_preds += torch.sum(preds == targets)
          losses.append(loss.item())
    val_acc = correct_preds / len(data['validation'])
    val_loss = np.mean(losses)
    iter_time = time.time() - start
    print(f"epoch {epoch + 1} -- train accuracy: {train_acc}, train loss: {train_loss}, validation accuracy: {val_acc}, validation loss: {val_loss}, epoch time: {int(iter_time)} (s)")

In [ ]:
def eval(model, test_dataloader):
  model.eval()
  losses = []
  correct_preds = 0
  with torch.no_grad():
    for d in tqdm(test_dataloader):
      input_ids = d[0].to(device)
      attention_mask = d[1].to(device)
      targets = d[2].to(device)
      outputs = model(input_ids, attention_mask=attention_mask, labels=targets)
      _, preds = torch.max(outputs.logits, dim=1)
      loss = outputs.loss
      correct_preds += torch.sum(preds == targets)
      losses.append(loss.item())
  test_acc = correct_preds / len(data['test'])
  test_loss = np.mean(losses)
  print(f"test accuracy: {test_acc}, test loss: {test_loss}")

In [ ]:
data = load_dataset('persiannlp/parsinlu_entailment')
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
labels = {'c': 0, 'e': 1, 'n': 2}
max_len = 256
batch_size = 8
train_dataloader = create_data_loader(data['train'], labels, tokenizer, max_len, batch_size)
test_dataloader = create_data_loader(data['test'], labels, tokenizer, max_len, batch_size)
validation_dataloader = create_data_loader(data['validation'], labels, tokenizer, max_len, batch_size)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1675 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/270 [00:00<?, ? examples/s]

Dataset parsinlu_reading_comprehension downloaded and prepared to /root/.cache/huggingface/datasets/persiannlp___parsinlu_reading_comprehension/parsinlu-repo/1.0.0/a99ad431d1637751f03fa9da3de66a41e93983d11116a0918a6efaaa72343d2d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [11]:
model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=3)
model.to(device)
train(model, train_dataloader, validation_dataloader)
eval(model, test_dataloader)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

epoch 1 -- train accuracy: 0.38675495982170105, train loss: 1.091328980420765, validation accuracy: 0.2888888716697693, validation loss: 1.1256644129753113, epoch time: 41 (s)


100%|██████████| 34/34 [00:03<00:00,  8.98it/s]


epoch 2 -- train accuracy: 0.3947019875049591, train loss: 1.0900035657380756, validation accuracy: 0.2888888716697693, validation loss: 1.1256409778314478, epoch time: 42 (s)


100%|██████████| 34/34 [00:03<00:00,  8.96it/s]


...

epoch 9 -- train accuracy: 0.3907284736633301, train loss: 1.0915520642933092, validation accuracy: 0.2888888716697693, validation loss: 1.1244431558777304, epoch time: 44 (s)


100%|██████████| 34/34 [00:03<00:00,  8.69it/s]


epoch 10 -- train accuracy: 0.3920529782772064, train loss: 1.0941699636609932, validation accuracy: 0.2888888716697693, validation loss: 1.1278647885603064, epoch time: 44 (s)


100%|██████████| 210/210 [00:24<00:00,  8.66it/s]

test accuracy: 0.3641791045665741, test loss: 1.1027888715267182


In [12]:
data = load_dataset('persiannlp/parsinlu_entailment')
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
labels = {'c': 0, 'e': 1, 'n': 2}
max_len = 256
batch_size = 8
train_dataloader = create_data_loader(data['train'], labels, tokenizer, max_len, batch_size)
test_dataloader = create_data_loader(data['test'], labels, tokenizer, max_len, batch_size)
validation_dataloader = create_data_loader(data['validation'], labels, tokenizer, max_len, batch_size)

Reusing dataset parsinlu_reading_comprehension (/root/.cache/huggingface/datasets/persiannlp___parsinlu_reading_comprehension/parsinlu-repo/1.0.0/a99ad431d1637751f03fa9da3de66a41e93983d11116a0918a6efaaa72343d2d)


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/434 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

In [13]:
model = AutoModelForSequenceClassification.from_pretrained('HooshvareLab/bert-base-parsbert-uncased', num_labels=3)
model.to(device)
train(model, train_dataloader, validation_dataloader)
eval(model, test_dataloader)

Downloading:   0%|          | 0.00/624M [00:00<?, ?B/s]

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifica

epoch 1 -- train accuracy: 0.3920529782772064, train loss: 1.0955917534075286, validation accuracy: 0.3592592477798462, validation loss: 1.1324022692792557, epoch time: 37 (s)


100%|██████████| 34/34 [00:03<00:00,  8.57it/s]


epoch 2 -- train accuracy: 0.565562903881073, train loss: 0.944258221199638, validation accuracy: 0.36666667461395264, validation loss: 1.1124562726301306, epoch time: 38 (s)


...

epoch 9 -- train accuracy: 0.9960265159606934, train loss: 0.022713756041699333, validation accuracy: 0.47777777910232544, validation loss: 2.0913224325460544, epoch time: 40 (s)


100%|██████████| 34/34 [00:04<00:00,  8.00it/s]


epoch 10 -- train accuracy: 0.9947019815444946, train loss: 0.02021483035120917, validation accuracy: 0.4888888895511627, validation loss: 2.378786521799424, epoch time: 40 (s)


100%|██████████| 210/210 [00:26<00:00,  7.99it/s]

test accuracy: 0.4591044485569, test loss: 2.3406931464515983
